<a href="https://colab.research.google.com/github/JoungMinJu/lab_study/blob/main/%EC%88%9C%ED%99%98%EC%8B%A0%EA%B2%BD%EB%A7%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.models import Sequential
from keras.layers import SimpleRNN


In [3]:
model=Sequential()
model.add(SimpleRNN(3, input_shape=(2,10)))
#hidden size, timesteps(=input_length), input_dim

model.summary()

#출력값은 (batch_size, output_dim)의 2D 텐서인데 현 상태에서 배치사이즈 알 수 없으므로 None, 3이 된다.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [4]:
# 배치사이즈 미리 정의
model=Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10)))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (8, 3)                    42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


In [5]:
#출력값으로 batch_size,timesteps,output_dim의 3D 텐서를 리턴하게 하겠다.
model=Sequential()
model.add(SimpleRNN(3, batch_input_shape=(8,2,10), return_sequences=True))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (8, 2, 3)                 42        
Total params: 42
Trainable params: 42
Non-trainable params: 0
_________________________________________________________________


직접 파이썬으로 RNN 구현해보기

촉 은닉상태를 0벡터로 초기화하고 각 시점마다 입력을 받으면서 각 시점에 대해 입려과 은닉상태를 가지고 연산하고 계산 결과는 현재 시점의 은닉상태가 되도록 구현하면 된다.

t시점의 은닉상태는 hidden_state_t이고 입력데이터의 길이는 input_length, 이 경우 입력 데이터의 길이는 곧 총 시점의 수 timesteps가 된다. 그리고 t시점의 입력값을 input_t 로 선언.

각 메모리 셀은 각 시점마다 input_t와 hidden_state_t(이전 상태의 은닉상태)를 입력으로 활성화함수인 하이퍼볼릭탄젠트 함수를 ㅌㅇ해 현 시점의 hidden_state_t를 계산한다.



In [7]:
import numpy as np 
#쉽게하려고 (timesteps, input_dim)으로 받았는데 실제 케라스에서는 (batch_size, timesteps, input_dim)으로 받는다

timesteps=10 #시점의 수, NLP에서는 보통 문장의 길이
input_dim=4 #입력의 차원, NLP에서는 보통 단어 벡터의 차원
hidden_size=8 # 은닉상태의 크기, 메모리 셀의 용량

inputs=np.random.random((timesteps, input_dim))#입력에 해당되는 2D 텐서
#0~1보다 작은 값의 범주에서 사이즈만큼 return gksek. ((3,2))면 ((3,2))사이즈로 return

hidden_state_t=np.zeros((hidden_size,)) #초기 은닉상태는 0벡터로 초기화 
#은닉상태의 크기로 은닉상태를 만듬


In [8]:
print(hidden_state_t)

[0. 0. 0. 0. 0. 0. 0. 0.]


In [9]:
#가중치와 편향 정의
Wx=np.random.random((hidden_size, input_dim))
Wh=np.random.random((hidden_size, hidden_size))
b=np.random.random((hidden_size,))

In [11]:
print(np.shape(Wx)) #은닉상태의 크기 * 입력의 차원
print(np.shape(Wh)) #은닉상태의 크기 * 은닉상태의 크기
print(np.shape(b)) #은닉상태의 크기

(8, 4)
(8, 8)
(8,)


In [15]:
#모든 시점의 은닉상태 출력한다고 가정하고 RNN 층을 동작시킴
total_hidden_states=[]
#메모리 셀 동작
for input_t in inputs: #각 시점에 따라서 입력값이 입력됨
  output_t=np.tanh(np.dot(Wx, input_t)+np.dot(Wh, hidden_state_t)+b)
  total_hidden_states.append(list(output_t)) #각 시점의 은닉상태값을 계속해서 축적
  print(np.shape(total_hidden_states))#각 시점 t별 메모리 셀의 출력 크기는 (timesteps output_dim)
  hidden_state_t=output_t
total_hidden_states=np.stack(total_hidden_states,axis=0) #출력 좀 깔끔히
print(total_hidden_states)

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
[[0.99998618 0.99993596 0.99999858 0.99999093 0.99994482 0.99999766
  0.99999062 0.99997908]
 [0.99995974 0.99978326 0.99999474 0.99997315 0.99980126 0.99999176
  0.99999105 0.99996299]
 [0.99995791 0.99986733 0.99999662 0.99998377 0.9998741  0.99999385
  0.99999408 0.99997958]
 [0.99990771 0.99968959 0.99999309 0.99996949 0.99978724 0.99998801
  0.99997931 0.99991945]
 [0.99997949 0.99991698 0.99999801 0.99998864 0.99992207 0.9999966
  0.99999337 0.99998196]
 [0.99995226 0.99970127 0.99999485 0.9999676  0.9998203  0.99999291
  0.9999713  0.99989561]
 [0.99995609 0.99988856 0.99999753 0.99998712 0.99991109 0.99999542
  0.99999214 0.99997731]
 [0.9998301  0.99965673 0.99999311 0.99997103 0.99980437 0.99998698
  0.99997541 0.99990802]
 [0.99991912 0.9997048  0.9999947  0.99997225 0.99984247 0.99999144
  0.99995956 0.99986967]
 [0.999979   0.99990687 0.99999759 0.99998695 0.99990381 0.99999589
  0.99999441 0.99998306]]